In [ ]:
paper = """Hawking for everyone Commemorating half a
century of an unfinished scientific revolution
Jorge Pinochet
Centro de Investigaci on en Educaci on 
N ucleo Pensamiento Computacional y Educaci on para el Desarrollo Sostenible 
Universidad Metropolitana de Ciencias de la Educaci on
Av Jos e Pedro Alessandri 774 Nu noa Santiago Chile
email jorgepinochetumcecl
This year marks half a century since Stephen Hawking made his greatest scientific dis
covery by theoretically proving that black holes aint so black as they behave like hot
bodies with an absolute temperature that depends inversely on their mass This discovery
is expressed by a simple and elegant equation known as the Hawking temperature The
best way to commemorate this great scientific event is by bringing it to a wide audience
The simplest and most transparent and intuitive tool to achieve this goal is dimensional
analysis The objective of this work is to use this tool to derive the Hawking equation
reveal its meaning and explore its main physical consequences
Keywords  Black holes Hawking temperature dimensional analysis
This year marks half a century since Stephen Hawking revolutionised physics by proposing
in 1974 that black holes aint so black  Specifically Hawking theoretically
demonstrated that a black hole emits thermal radiation as if it were a hot body with an
absolute temperature TH known as the Hawking temperature  which can be calculated
by the equation
THc3
where MBHis the mass of the black hole and  c k G are fundamental physical constants
Despite the scientific relevance of Hawking temperature and the time that has passed since it
was formulated it continues to be unknown and at best misunderstood by the vast majority
of people
The best way to commemorate the 50th anniversary of Hawkings discovery is to bring it
to a wide audience The simplest and most transparent and intuitive tool for this task is
dimensional analysis  the application of which requires a minimal knowledge of physics
and mathematics In this work we will use dimensional analysis to carry out an original
derivation of the Hawking temperature Following this we will explore the meaning of this
equation and its main physical consequences namely that black holes evaporate and have
entropy
This manuscript may be useful to gifted high school students physics teachers scientists and
engineers and more generally to any educated person who is interested in learning about
Stephen Hawkings most important scientific contribution
Figure 1 Stephen Hawking born in 1942 was a renowned British theoretical physicist known for his
contributions to cosmology and black hole physics His greatest scientific discovery was the equation
that established that black holes behave as hot bodies with an absolute temperature that depends
inversely on their mass Despite facing progressive motor paralysis caused by Lou Gehrigs disease
Hawking continued to research and communicate his revolutionary ideas until his death in 2018
Black holes are the most extreme prediction of general relativity which is the theory of gravity
proposed by Einstein to refine and extend Newtons law of gravitation  A black hole is a
region of space where there is such a high concentration of matter that nothing can escape
its powerful gravity not even light  The simplest of these objects is the socalled static
black hole whose mathematical description depends only on its mass Eq  applies to
these objects so we will focus on them here
Fig 2 shows an intuitive representation of a static black hole  The mass of this object is
concentrated in a point region called a singularity  located in the centre of the event horizon 
or simply horizon  which is a spherical surface that defines the outer limit of the black hole
Although the horizon has no material existence it can be imagined as a unidirectional mem
brane that only allows matter or energy to flow inward  The radius of the horizon called
thegravitational radius  is calculated as 
Rg2GM BH
c2 
where Gis the gravitational constant cis the speed of light in a vacuum and MBHis the
mass of the black hole that is the mass confined in the singularity Since nothing can cross
the horizon to the outside this region appears completely black If the description provided
by general relativity is correct we see that the laws of thermodynamics ensure that the tem
perature of the horizon is strictly zero otherwise the black hole would emit thermal radiation
and would not be black
Figure 2 Structure of a static black hole
Broadly speaking this is the classic black hole paradigm that emerges from general relativ
ity In 1974 when Hawking entered the scene no one questioned this paradigm However
Hawking dared to question it proving that black holes aint so black Specifically Hawking
proved that the horizon behaves as a hot body with an absolute temperature TH the Hawking
temperature  which is proportional to the gravity at the horizon
Figure 3 Black holes emit thermal radiation have entropy and gradually evaporate
Table 1 Theories and physical constants involved in the Hawking temperature
Theory Characteristic constant Name
General relativityG 6671011Nm2kg2Gravitational constant
c 3108ms1Speed of light in vacuum
Quantum mechanics  1051034kgm2s1Planck constant
Thermodynamics k 1381023kgm2K1Boltzmann constant
As a consequence the horizon emits thermal radiation in all directions the socalled Hawking
radiation  According to Einsteins massenergy equivalence the energy radiated
by the black hole leads to a gradual reduction of MBHandRSthrough a process called
evaporation  Furthermore from the laws of thermodynamics this implies that a black
hole has entropy which turns out to be proportional to the area of the horizon From Eq
ABH 4R2
g16G2M2
c4 
To reach these conclusions Hawking  combined general relativity which describes
the universe at a macroscopic level with quantum mechanics which describes the universe at
a microscopic level and thermodynamics which describes thermal phenomena  Table 1
shows the fundamental physical constants that characterise these theories which will be very
useful when dimensionally deriving the Hawking temperature
Table 2 SI units and their corresponding dimensions
Dimension Symbol SI unit SI symbol
Mass M kilogram kg
Lenght L metre m
Time T second s
Temperature  kelvin K
To facilitate the derivation which is presented in the next section the constants in Table
showing the dimensions that are relevant for our purposes along with their symbols and the
associated SI units To guide readers who are not familiar with dimensional analysis an
appendix has been included that summarises the main concepts of this tool
All the physics we need for a dimensional derivation of the Hawking temperature is sum
marised in Tables 1 and 2 and in the idea that the description of a static black hole depends
only on its mass We will start by assuming that we do not know the Hawking equation Ob
viously our ignorance on the subject cannot be complete in fact to carry out a dimensional
derivation it is always necessary to start with a hypothesis about the dimensional parameters
involved  
To formulate our hypothesis we must keep in mind that Hawkings great achievement was to
combine general relativity with quantum mechanics and thermodynamics to demonstrate that
a static black hole has an absolute temperature TH As shown in Table 1 this suggests that
THdepends on the characteristic constants of these three theories G c  k On the other
hand we also know that a static black hole is completely characterised by its mass MBH
This shows us that a total of six dimensional parameters are involved TH G c  k M BH
and four dimensions M L T  However we can reduce these parameters to five and thus
simplify the dimensional derivation of TH by introducing a new variable which in astrophysics
is known as the standard gravitational parameter  and for a celestial body of mass mis defined
asGm The usefulness of this parameter lies in the fact that in problems where gravity
applies as is the case with Hawkings discovery the product Gmappears frequently1
that the use of is essential to this derivation Indeed with p 6 parameters  and d 4
dimensions  the number of dimensionless groups or Pi groups that can be formed is dp 2
Table 3 Dimensional parameters symbols and dimensions
Parameter Symbol Dimensions
Hawking temperature TH M0L0T01
Speed of light in vacuum c M0L1T10
Planck constant  M1L2T10
Boltzmann constant k M1L2T21
Standard gravitational parameter M0L3T20
Under these conditions if we define the standard gravitational parameter as GM BH we
can formulate our hypothesis in the form
Hypothesis THf
The specific information needed to derive the Hawking equation is summarised in Table 3
which contains the dimensional parameters their symbols and the corresponding dimensions
We can rewrite our hypothesis in the form
TH cx1x2kx3x4 
where  is an indeterminate dimensionless quantity The exponents x1 x2 x3andx4are
unknown and our main objective is to determine their numerical values Starting from the
rules of dimensional algebra and remembering that the notation  means the dimension of
x  we can rewrite Eq  as
Since  is dimensionless it must be true that   1 By introducing the dimensions of the
five parameters listed in Table 3 we obtain
M0L0T01
M0L1T10x1
M1L2T10x2
M1L2T21x3
M0L3T20x4
After sorting and grouping terms we have
M0L0T01M0x1x2x30x4Lx12x22x33x4Tx1x22x32x40x10x2x30x4
To satisfy this equality each exponent on the right side must be equal to the corresponding
exponent on the left side producing a linear system of four equations with four unknowns
x1 2x2 2x3 3x4 0
which implies that the equation for THremains undetermined However with the introduction of  we have
d 5 parameters and since the number of dimensions remains d 4 the number of Pi groups is reduced to
dp 1 and the equation for THis uniquely determined
By substitution the reader can verify that the solution of this system is x1 3 x2 1 x3
TH c3
k c3
kGM BH 
Except for the dimensionless parameter  this equality is identical to Eq  Comparing
Eqs  and  we see that   1 8 The interested reader can find other simple derivations
of the Hawking temperature in 
Hawking temperature is not an isolated result as it has two physical implications that continue
to raise questions the first is that black holes evaporate and the second is that they have
entropy To understand these two important results in general terms we must begin by
exploring the physical meaning of the Hawking temperature
Table 4 The Hawking effect
Discovery Mathematical formulation Description
Hawking temperature THc3
inversely on their mass MBH The greater the value of MBH
the lower the value of TH and vice versa For known black
holes THis undetectable
Hawking radiation and
evaporationtev103G2M3
c4Black holes evaporate in a characteristic time tevthat de
pends on the cube of their mass For known black holes tev
is much older than the age of the universe and evaporation
is undetectable
BekensteinHawking
entropySBHkc3ABH
cBlack holes have an entropy SBHthat depends on the hori
zon area ABH which is proportional to the square of the
mass The maximum entropy that can accumulate in a re
gion of fixed size is that of the largest black hole confined in
said region
Table 4 summarises the equations that describe the Hawking effect  which is the name given
to the set of phenomena associated with Hawkings discovery The interested reader will find
very simple derivations of these equations in  In the following sections we will delve into
the Hawking effect Due to space limitations we will not analyse the physical mechanism
responsible for the Hawking temperature A reader interested in this topic can refer to
Hawkings texts  or other works published in this journal where this topic is addressed
in an accessible way 
The first aspect that we must keep in mind in relation to the Hawking temperature is that it
does not depend on the interaction of the black hole with the matter in its environment A
black hole could be completely isolated and still have a temperature given by Eq 
We will start by rewriting this equation in a form that allows for quick calculations using the
values of G c  kfrom Table 1 If we enter the solar mass M 1991030kg and multiply
the right side of Eq  by the dimensionless quotient M1991030kg 1 we obtain the
Hawking temperature expressed in kelvin  as
TH 617108KM
MBH
where THis the temperature of an isolated black hole that is a black hole where the outer
region of its horizon contains no matter or radiation This equation tells us that black holes
aint so black and that they can be very bright and hot if MBHis small enough To provide
empirical corroboration of Hawkings discovery we need to find light black holes The lightest
ones that have been observed so far have masses of the order of M while the heaviest
ones can have masses of up to 1010M To obtain an upper bound for TH we take
MBHMin Eq  obtaining TH 617108K This tiny temperature is undetectable
through astronomical observations
Figure 4 Graph of Hawking temperature TH versus MBHM
To illustrate the relationship between THandMBHmore clearly a graph of Eq  is shown
in Fig 4 The horizontal axis has dimensionless units of MBHMand the vertical axis has
units of 6 17108K It can be observed that THapproaches zero asymptotically as MBH
increases
As with any hot body a black hole emits mainly photons that is Hawking radiation is ba
sically light However if THis high enough it can also emit neutrinos electrons positrons
etc This emission process is known as evaporation  Just as the evaporation of a puddle
of water on a summer day gradually reduces the mass of the puddle Hawking radiation grad
ually reduces the mass of the black hole as the photons electrons and other particles cross
the horizon to the outside
Eq  reveals that the smaller the value of MBH the more Hawking radiation is emitted
from the horizon and the more radiation is emitted the more rapidly MBHis reduced This
means that evaporation does not occur at a constant rate but instead increases over time
when they emit more material than they absorb that there will there be a net outward flow
of radiation that will reduce MBH The time taken for the mass of an isolated black hole of
initial mass MBHto be reduced to zero is called the evaporation time 
tev103G2M3
c4 
If we introduce the values of G c and multiply the right side of this equality by the dimen
sionless quotient 3 1 the evaporation time is expressed in seconds 
as
tev 1073sMBH
M3
Fig 5 shows a graph of this equation where the horizontal axis has units of MBHMand
the vertical axis has units of 1073s It can be observed that tevincreases rapidly with an
increase in MBH
Eq  reveals that in general tevis extraordinarily large To obtain a lower bound for tev
we take MBHMin Eq  obtaining tev1073s This figure is 56 orders of magnitude
greater than the age of the universe which is estimated at 1017s Hence evaporation is
unobservable which is consistent with the fact that THis undetectable
Figure 5 Graph of evaporation time tev versus MBHM
The laws of thermodynamics establish that every body with a nonzero temperature has
entropy and black holes are no exception According to calculations originally carried out
by the physicist Jacob Bekenstein and perfected by Hawking  the BekensteinHawking
entropy SBHof an isolated black hole is proportional to the area of its event horizon
SBHkc3ABH
Since ABHis given by Eq  the entropy can also be written as a function of the mass of
the black hole and the solar mass
SBH4kGM2
c151054JK1MBH
M2
where we have introduced the values of the constants and multiplied the right side of the
equality by the dimensionless quotient 2 1 allowing SBHto be expressed
in jouleskelvin  which are the SI units of entropy Fig 6 shows a graph of this
equation where the horizontal axis has units of MBHMand the vertical axis units of
gives SBH 151054JK1 This is a colossal number and reveals that black holes are
among the most entropic objects in the universe
Figure 6 Graph of entropy SBH versus MBHM
If as the evidence suggests the smallest black holes have masses on the order of M with
Hawking temperatures on the order of a millionth of a kelvin attempts to detect Hawking
radiation may be unsuccessful To get around this difficulty the ideal would be to create
black holes in our laboratories which would allow us to study them in detail but this is
obviously impossible with current technology  and it may never be possible to develop
such technology
Due to these difficulties some specialists have turned their attention towards a field of ex
perimental physics called analogue gravity  whose objective is implementing physical systems
that imitate the behaviour of gravitational phenomena that are difficult to observe under real
conditions  such as the Hawking effect
The first person to propose the use of analogue models to study black holes was the Canadian
theoretical physicist William Unruh who in 1972 introduced an inventive analogy he imag
ined a fish screaming while falling over a waterfall As the fish falls it heads toward a point in
the waterfall where the water travels downward faster than sound waves can travel upward
When the fish crosses this point its friends in the river above can no longer hear it since
to enable this the speed of the sound waves would have to exceed the speed of sound in the
water  According to Unruh this is analogous to what would happen to an astronaut
who fell into the jaws of a black hole the astronaut would not be able to communicate with
the base station on Earth because the signals sent towards the outside would need to move
faster than light which is prohibited by physical laws
Figure 7 A fish sends sound waves up a waterfall The dashed line is an analogue of the horizon
where the speed with which the waterfall falls is equal to the speed of sound in the water meaning
that the waves cannot cross the dashed line
Almost a decade later Unruh showed that the event horizons of black holes and the sonic
horizons in systems such as his waterfall which are now known as sonic black holes  can be
described by identical equations  In particular Unruh showed that sonic black holes can
mimic the Hawking temperature and Hawking radiation The results of the first experiment
of this type were published in 2014  so this is a new and developing field of research The
results obtained so far are encouraging although not conclusive and the evidence is consistent
with the discoveries made by Hawking half a century ago
The great discoveries of physics are expressed through equations that accurately describe a
wide range of phenomena Some equations stand out for their simplicity others for their
elegance and others for their ability to build bridges between apparently dissimilar domains
of reality However there is a select group of equations that stand out for combining all these
virtues
For example Newtons law of gravitation falls into this category as it reconciles celestial
physics with terrestrial physics as does Einsteins massenergy equivalence which unifies
matter and energy The Hawking equation also falls into this category as it bridges general
relativity quantum mechanics and thermodynamics Hawkings tomb in Westminster Abbey
has this equation engraved on it alongside the inscription Here lies what was mortal of
Stephen Hawking It is hoped that this work will contribute to a better understanding of
Hawkings work and his immortal equation a symbol of a scientific revolution that is far from
over
Appendix Dimensional analysis and dimensional algebra
Aphysical dimension  briefly called a dimension  is a property of an object or phenomenon
that can be measured and expressed in terms of standard fundamental units Dimensional
analysis is a tool used in science and engineering to study and relate different physical quanti
ties through their dimensions For example length mass and time are dimensions that have
associated SI units of the metre kilogram and second respectively There are seven dimen
sions which correspond to the seven fundamental physical magnitudes These dimensions
their symbols and the associated SI units appear in Table A1 below
In the various applications of dimensional analysis the notation  is commonly used which
is read as the dimensions of a For example if mis the mass of any object is its density
andvis its speed we have  M  ML3and  LT1
Table A1  SI units and their corresponding dimensions
Dimension Symbol SI unit SI symbol
Mass M kilogram kg
Lenght L metre m
Time T second s
Temperature  kelvin K
Electric current I ampere A
Luminous intensity J candle Cd
Amount of substance N mole mol
To apply dimensional analysis it is useful to systematise seven principles that together we
can call the rules of dimensional algebra  We can consider a set of quantities a b c that
may be pure numbers variables physical constants etc The rules of dimensional algebra
that can be applied to these quantities are as follows
iProduct rule    
iiQuotient rule    
iiiCommutativity rule    
ivAssociativity rule    
vPowers rule    n
viHomogeneity rule  Ifabcthen       
viiDimensionless quantities rule  Ifais dimensionless then   1
I would like to thank to Daniela Balieiro for their valuable comments in the writing of this
paper"""

In [ ]:
summary = """The paper commemorates the 50th anniversary of Stephen Hawking's groundbreaking discovery of black holes emitting thermal radiation, known as Hawking radiation. The authors use dimensional analysis to derive the Hawking temperature, a fundamental concept in modern astrophysics. They demonstrate that black holes have an absolute temperature, TH, which depends inversely on their mass, MBH. The authors also explore the physical implications of Hawking's discovery, including the evaporation of black holes and their entropy.

The paper begins by introducing the concept of dimensional analysis, a tool used to study physical quantities with different dimensions. The authors then derive the Hawking temperature using dimensional analysis, starting from the assumption that a static black hole is characterized by its mass. They introduce the standard gravitational parameter, Gm, which is essential to the derivation.

The authors then discuss the physical meaning of Hawking's discovery, highlighting the implications of black holes having an absolute temperature. They also explore the concept of entropy, which is proportional to the area of the event horizon. The paper concludes by discussing the challenges of detecting Hawking radiation and the potential applications of analogue gravity, a field that aims to create physical systems that mimic the behavior of gravitational phenomena.

Throughout the paper, the authors use a range of technical terms, including dimensional analysis, Hawking radiation, entropy, and analogue gravity. They also provide a detailed derivation of the Hawking temperature, using mathematical equations and formulas. The paper is written in a clear and concise style, making it accessible to readers with a background in astrophysics and physics.

"""

In [ ]:
from typing import Dict, List
import numpy as np
from transformers import AutoTokenizer, AutoModel
from rouge_score import rouge_scorer
from bert_score import BERTScorer
import torch
import re

class ScientificMetricsEvaluator:
    def __init__(self):
        """Initialize scientific metrics"""
        self.rouge_scorer = rouge_scorer.RougeScorer(
            ['rouge1', 'rouge2', 'rougeL', 'rougeLsum'],
            use_stemmer=True
        )
        
        self.bert_scorer = BERTScorer(
            model_type="adsabs/astroBERT",
            num_layers=9,
            batch_size=32,
            nthreads=4,
            all_layers=False,
            idf=False,
            lang='en',
            rescale_with_baseline=False
        )
        
        self.tokenizer = AutoTokenizer.from_pretrained("adsabs/astroBERT")
        self.max_length = 512

    def _preprocess_text_rouge(self, text: str) -> str:
        """Preprocessing for ROUGE - keeps whitespace"""
        text = re.sub(r'[^\w\s,.!?-]', '', text)
        return ' '.join(text.split()).strip()

    def _preprocess_text_bert(self, text: str) -> str:
        """Minimal preprocessing for BERT - only removes potentially problematic characters"""
        return re.sub(r'[^\w,.!?-]', '', text)

    def _truncate_text(self, text: str) -> str:
        """Truncate text to fit within BERT's maximum sequence length"""
        tokens = self.tokenizer.encode(text, add_special_tokens=True, max_length=self.max_length, truncation=True)
        return self.tokenizer.decode(tokens)

    def calculate_rouge(self, reference: str, candidate: str) -> Dict[str, float]:
        """Calculate ROUGE scores"""
        reference = self._preprocess_text_rouge(reference)
        candidate = self._preprocess_text_rouge(candidate)
        
        scores = self.rouge_scorer.score(reference, candidate)
        
        return {
            'rouge1': scores['rouge1'].fmeasure,
            'rouge2': scores['rouge2'].fmeasure,
            'rougeL': scores['rougeL'].fmeasure,
            'rougeLsum': scores['rougeLsum'].fmeasure
        }

    def calculate_bertscore(self, reference: str, candidate: str) -> Dict[str, float]:
        """Calculate BERTScore using truncated texts"""
        reference = self._preprocess_text_bert(reference)
        candidate = candidate
        
        reference = self._truncate_text(reference)
        candidate = candidate
        
        P, R, F1 = self.bert_scorer.score([candidate], [reference])
        
        return {
            'precision': float(P[0]),
            'recall': float(R[0]),
            'f1': float(F1[0])
        }

    def calculate_ngram_novelty(self, reference: str, candidate: str) -> float:
        """Calculate the proportion of novel n-grams in the candidate"""
        reference = self._preprocess_text_rouge(reference)
        candidate = candidate
        
        def get_ngrams(text, n):
            words = text.split()
            return set(' '.join(words[i:i+n]) for i in range(len(words)-n+1))
        
        # Calculate for different n-gram sizes
        novelty_scores = []
        for n in [1, 2, 3]:
            ref_ngrams = get_ngrams(reference, n)
            cand_ngrams = get_ngrams(candidate, n)
            novel_ngrams = cand_ngrams - ref_ngrams
            if cand_ngrams:
                novelty_scores.append(len(novel_ngrams) / len(cand_ngrams))
        
        return np.mean(novelty_scores) if novelty_scores else 0.0

    def evaluate_summary(self, reference: str, candidate: str) -> Dict[str, float]:
        """Calculate all metrics and combine them with weights optimized for abstractive summaries"""
        rouge_scores = self.calculate_rouge(reference, candidate)
        bert_scores = self.calculate_bertscore(reference, candidate)
        novelty_score = self.calculate_ngram_novelty(reference, candidate)
        
        final_scores = {
            'rouge1': rouge_scores['rouge1'],
            'rouge2': rouge_scores['rouge2'],
            'rougeL': rouge_scores['rougeL'],
            'rougeLsum': rouge_scores['rougeLsum'],
            'bertscore_precision': bert_scores['precision'],
            'bertscore_recall': bert_scores['recall'],
            'bertscore_f1': bert_scores['f1'],
            'ngram_novelty': novelty_score
        }
        
        # New weighting scheme that rewards:
        # 1. High semantic similarity (BERTScore)
        # 2. High n-gram novelty
        # 3. Moderate ROUGE-L (some structure preservation)
        weights = {
            'bertscore_f1': 0.5,      # Semantic preservation
            'ngram_novelty': 0.3,     # Vocabulary diversity
            'rougeL': 0.2             # Basic structure preservation
        }
        
        final_scores['abstractive_score'] = sum(
            final_scores[metric] * weight
            for metric, weight in weights.items()
        )
        
        return final_scores

def evaluate_scientific_summary(reference_text: str, summary_text: str) -> Dict[str, float]:
    """Convenience function for evaluating scientific summaries"""
    evaluator = ScientificMetricsEvaluator()
    return evaluator.evaluate_summary(reference_text, summary_text)

In [16]:
scores = evaluate_scientific_summary(paper, summary)
print(scores)

AttributeError: 'ScientificMetricsEvaluator' object has no attribute 'candidate'